# Image Extractor
This notebook can iterate over an existing bag and save images from the RGB camera into a pre-specified folder. This is useful or ODM to speed up the matching process.

In [ ]:
%load_ext autoreload
%autoreload 2
    
import sys
    
sys.path.append('..')
import ha_python_utils.constants as const
from  ha_python_utils.bag_reader import BagReader
from rclpy.serialization import serialize_message
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from pathlib import Path
import rosbag2_py
from event_camera_py import Decoder
import copy
import yaml
import time
import collections
import cv2
import shutil
import pdb
import utm
import glob
from os.path import basename, join, exists

In [ ]:
# Input synchronized bag
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/cam.calib.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/cam.calib.2/synchronized/synchronized_0.mcap")
SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/flight.day.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/flight.day.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/imu.calib.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.19.Pennov.Flight.and.Calib/imu.calib.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/cam.calib.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/cam.calib.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.3/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.day.4/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/flight.night.3/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.1/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.2/synchronized/synchronized_0.mcap")
# SYNCED_BAG = Path("/data/ha_ec_data/2024.08.25.Pennov.Flight.and.Calib/imu.calib.3/synchronized/synchronized_0.mcap")

In [ ]:
# Get output folder from input bag
RGB_IMAGE_FOLDER = SYNCED_BAG.parents[1] / "images" / "RGB"

# Create dir if it does not exist
RGB_IMAGE_FOLDER.mkdir(parents=True, exist_ok=True)

# Delete all content in the folder 
shutil.rmtree(RGB_IMAGE_FOLDER)
RGB_IMAGE_FOLDER.mkdir(parents=True, exist_ok=True)

E2VID_IMAGE_INPUT = SYNCED_BAG.parents[1]/"extracted_events/e2vid/reconstruction"
E2VID_TIMESTAMPS = E2VID_IMAGE_INPUT/"timestamps.txt"

E2VID_PROCESSED_IMAGE_FOLDER = SYNCED_BAG.parents[1]/"images/E2VID_PROCESSED"
E2VID_RAW_IMAGE_FOLDER = SYNCED_BAG.parents[1]/"images/E2VID_RAW"
RECON_RAW_IMAGE_FOLDER = SYNCED_BAG.parents[1]/"images/RECON_RAW"

# Check that the timestamp files exists and load into array
if E2VID_TIMESTAMPS.exists() and E2VID_TIMESTAMPS.is_file():
    stamps = np.loadtxt(E2VID_TIMESTAMPS)
    stamps_sorted = np.sort(stamps)
    assert np.all(np.isclose(stamps_sorted - stamps, 0))
    images = np.array([f for f in glob.glob(join(E2VID_IMAGE_INPUT, "*.png"), recursive=False)])
    img_numbers = [int(Path(i).name.split("_")[1].split(".")[0]) for i in images]
    images = images[np.argsort(img_numbers)]
    assert len(stamps) == len(images)
else:
    sys.exit("Path for timestamp file does not exist")
    
# Delete all content in the output image folder 
for folder in [E2VID_PROCESSED_IMAGE_FOLDER, E2VID_RAW_IMAGE_FOLDER, RECON_RAW_IMAGE_FOLDER]:
    folder.mkdir(parents=True, exist_ok=True)
    shutil.rmtree(folder)
    folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# Get GPS and range measurements
bag_reader = BagReader(SYNCED_BAG, [const.OUTPUT_RANGE_TOPIC, const.OUTPUT_GPS_TOPIC])
gps_measurements = np.zeros((bag_reader.get_num_msgs(const.OUTPUT_GPS_TOPIC), 2), dtype=np.float64)
range_measurements = np.zeros(bag_reader.get_num_msgs(const.OUTPUT_RANGE_TOPIC), dtype=np.float64)
gps_cnt = 0
range_cnt = 0
for topic, msg, timestamp in bag_reader.read_all():
    if topic == const.OUTPUT_GPS_TOPIC:
        gps_measurements[gps_cnt, :] = np.array([msg.latitude, msg.longitude])
        # print(msg.latitude)
        gps_cnt += 1
    elif topic == const.OUTPUT_RANGE_TOPIC:
        range_measurements[range_cnt] = msg.range
        range_cnt += 1
        
# Detect once the UAV climbs to 20 m and when it descends below 20 m
logic_test = range_measurements > 20
idxs = np.where(logic_test)[0]
start_idx, end_idx = idxs[0], idxs[-1]
plt.figure()
plt.plot(range_measurements)
plt.plot(start_idx, range_measurements[start_idx], 'or',)
plt.plot(end_idx, range_measurements[end_idx], 'or')

In [ ]:
print(bag_reader.get_duration())
bag_reader.print_stats()
bag_reader = BagReader(SYNCED_BAG, [const.OUTPUT_RANGE_TOPIC, const.OUTPUT_GPS_TOPIC])
range_cnt = 0
for topic, msg, timestamp in bag_reader.read_all():
    print(timestamp)
    break

In [ ]:
# Match rgb image to event image
def crop_rgb_image(rgb_image):
    half = cv2.resize(rgb_image, (1450, 1088))
    cropped = half[168:168+720, 122:122+1280:]
    return cropped

# Get one image of each and exit. Test 
RECON_BAG = SYNCED_BAG.parents[1]/"simple_recon_bernd/simple_recon_bernd_0.mcap"
recon_reader = BagReader(RECON_BAG, [const.OUTPUT_RECON_TOPIC, const.OUTPUT_FLIR_IMAGE_TOPIC, const.OUTPUT_VNAV_IMU_TOPIC])
recon_img = None
rgb_img = None
for i, (topic, msg, timestamp) in enumerate(recon_reader.read_all()):
    if i <= 2000:
        continue
    if topic == const.OUTPUT_RECON_TOPIC:
        recon_img = cv2.cvtColor(np.frombuffer(msg.data, np.uint8).reshape((720, 1280)), cv2.COLOR_GRAY2RGB)
        recon_ts = timestamp
    elif topic == const.OUTPUT_FLIR_IMAGE_TOPIC:
        arr = np.frombuffer(msg.data, np.uint8)
        rgb_img = cv2.cvtColor(cv2.imdecode(arr, cv2.IMREAD_UNCHANGED), cv2.COLOR_BayerRG2RGB)
        rgb_ts = timestamp
    if recon_img is not None and rgb_img is not None:
        break
assert recon_ts == rgb_ts, f"recon_ts: {recon_ts} - rgb_ts: {rgb_ts}"
ts = (msg.header.stamp.sec*1000000000 + msg.header.stamp.nanosec)/1e9
idx = np.searchsorted(stamps, ts, side="left")
e2vid_img_path = Path(images[idx])
# rewrite the img_path with more leading zeros
img_number = int(e2vid_img_path.name.split("_")[1].split(".")[0])
new_e2vid_name = f"frame_{img_number:012}.png"
img_ev = cv2.imread(str(e2vid_img_path))

# Test the matching between RGB and events
blended_cut = crop_rgb_image(rgb_img)
blended_cut[:720//2, :1280//2, :] = img_ev[:720//2, :1280//2, :]
blended_cut[720//2:, 1280//2:, :]= img_ev[720//2:, 1280//2:, :]
# plt.imshow(rect_ev)
plt.imshow(blended_cut)
plt.show()

In [ ]:
bag_reader = BagReader(SYNCED_BAG, [const.OUTPUT_FLIR_IMAGE_TOPIC, const.OUTPUT_RANGE_TOPIC, const.OUTPUT_GPS_TOPIC])
recon_reader = BagReader(RECON_BAG, [const.OUTPUT_RECON_TOPIC])
utm_zone = 18
last_gps_msg = None
good_altitude = False
last_xyz = None
range_cnt = 0
with open(str(RGB_IMAGE_FOLDER/"geo.txt"), "w") as geo_rgb, open(str(E2VID_PROCESSED_IMAGE_FOLDER/"geo.txt"), "w") as geo_e2vid:
    geo_rgb.write("+proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs\n")
    geo_e2vid.write("+proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs\n")
    for topic, msg, timestamp in bag_reader.read_all():
        if topic == const.OUTPUT_RANGE_TOPIC:
            # Count the range messages
            if range_cnt < start_idx:
                range_cnt += 1
                continue
            elif range_cnt > end_idx:
                good_altitude = False
                break
            else:
                good_altitude = True
                range_cnt += 1
        elif topic == const.OUTPUT_GPS_TOPIC:
            # Get new lat-lon and set 
            last_gps_msg = msg
        elif topic == const.OUTPUT_FLIR_IMAGE_TOPIC and good_altitude and last_gps_msg is not None:
            # Get the UTM for the current gps coords
            easting, northing, zone_number, zone_letter = utm.from_latlon(last_gps_msg.latitude, last_gps_msg.longitude)
            assert zone_number == utm_zone # Check if we go fly somewhere else
            xyz = np.array([easting, northing, last_gps_msg.altitude])
            # Decide if we are taking a new image or not
            if last_xyz is not None:
                d = np.linalg.norm(xyz - last_xyz)
                if d < 2:
                    continue
                if d > 10:
                    print(f"Too big d: {d}")
            last_xyz = xyz
            # Find the matching recon image
            ts_recon = -1
            while ts_recon != timestamp and recon_reader.has_next():
                _, msg_recon, ts_recon = recon_reader.read_next()
            if ts_recon != timestamp:
                print("Recon image and rgb image do not have the same ts")
                break
            # Write the output rgb image, trying to mimic the EC output
            rgb_img_path = RGB_IMAGE_FOLDER / f"{timestamp:014}.png"
            arr = np.frombuffer(msg.data, np.uint8)
            img =  cv2.cvtColor(cv2.imdecode(arr, cv2.IMREAD_UNCHANGED), cv2.COLOR_BayerRG2RGB)
            cropped = crop_rgb_image(img)
            cv2.imwrite(str(rgb_img_path), cropped)
            geo_rgb.write(f"{rgb_img_path.name} {easting} {northing} {last_gps_msg.altitude}\n")
            
            # Find the closest e2vid image for the rgb image
            ts = (msg.header.stamp.sec*1000000000 + msg.header.stamp.nanosec)/1e9
            idx = np.searchsorted(stamps, ts, side="left")
            e2vid_img_path = Path(images[idx])
            # rewrite the img_path with more leading zeros
            img_number = int(e2vid_img_path.name.split("_")[1].split(".")[0])
            new_e2vid_name = f"frame_{img_number:012}.png"
            image = cv2.imread(str(e2vid_img_path))
            # Write the raw e2vid image
            cv2.imwrite(str(E2VID_RAW_IMAGE_FOLDER/new_e2vid_name), image)
            # CLAHE all the things
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            equalized = clahe.apply(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
            equalized = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
            cv2.imwrite(str(E2VID_PROCESSED_IMAGE_FOLDER/new_e2vid_name), equalized)
            geo_e2vid.write(f"{new_e2vid_name} {easting} {northing} {last_gps_msg.altitude}\n")

            # Write the recon image for the corresponding RGB
            arr = np.frombuffer(msg_recon.data, np.uint8).reshape(720, 1280)
            img_recon = cv2.cvtColor(arr, cv2.COLOR_GRAY2RGB)
            recon_img_path = RECON_RAW_IMAGE_FOLDER / f"{timestamp:014}.png"
            cv2.imwrite(str(recon_img_path), img_recon)
            
            print(f"ts: {ts} - idx: {idx:06} - e2vid_image: {new_e2vid_name} - rgb_image: {rgb_img_path.name}")
# Copy the geo.txt file from one folder to another one
shutil.copy(str(RGB_IMAGE_FOLDER/"geo.txt"), str(RECON_RAW_IMAGE_FOLDER/"geo.txt"))
shutil.copy(str(E2VID_PROCESSED_IMAGE_FOLDER/"geo.txt"), str(E2VID_RAW_IMAGE_FOLDER/"geo.txt"))